In [ ]:
%pip install confluent-kafka

In [3]:
import threading
import json
import time
import datetime
import random
from confluent_kafka import Producer

In [2]:

conf = {
    'bootstrap.servers': 'localhost:9092',  # change if using ngrok
    'client.id': 'iot-producer'
}

producer = Producer(conf)

In [ ]:
producer.flush()
print("Producer stopped cleanly.")


In [ ]:

conf = {
    'bootstrap.servers': '0.tcp.in.ngrok.io:12783',  # <--- ngrok host:port
    'client.id': 'iot-producer',
    'queue.buffering.max.messages': 200000,
}
producer = Producer(conf)
topic = "iot_signals"

# -----------------------
# Signal ranges
# -----------------------
signal_ranges = {}
for i in range(1, 11):
    signal_ranges[f"signal_{i}"] = (0, 1)
for i in range(11, 21):
    signal_ranges[f"signal_{i}"] = (0, 100)
for i in range(21, 41):
    signal_ranges[f"signal_{i}"] = (-50, 50)
for i in range(41, 61):
    signal_ranges[f"signal_{i}"] = (-100, 100)

# -----------------------
# Base timestamp
# -----------------------
base_time = int(datetime.datetime.now().timestamp() * 1_000_000)

# -----------------------
# Stop flag (set True from another cell to stop)
# -----------------------
stop_producer = False

def generate_row():
    global base_time
    step = random.randint(0, 5000)  # 0–5 ms
    base_time += step
    ts = datetime.datetime.fromtimestamp(base_time / 1_000_000).strftime("%m/%d/%Y %H:%M:%S:%f")
    row = {"timestamp": ts}
    for sig, (low, high) in signal_ranges.items():
        row[sig] = round(random.uniform(low, high), 4)
    return row

# -----------------------
# Producer loop
# -----------------------
target_rate = 10000  # msgs/sec
batch_size = 1000
interval = batch_size / target_rate
flush_every = 100000   # auto-flush every 100k msgs

print(f"Starting producer at ~{target_rate} msgs/sec...")

msg_count = 0
while not stop_producer:
    start = time.time()
    for _ in range(batch_size):
        row = generate_row()
        producer.produce(topic, value=json.dumps(row))
        producer.poll(0)   # prevents queue overflow
        msg_count += 1

        # Auto-flush safeguard
        if msg_count % flush_every == 0:
            producer.flush()
            print(f"Flushed at {msg_count} messages.")

    elapsed = time.time() - start
    if elapsed < interval:
        time.sleep(interval - elapsed)

# Final flush when stopping
producer.flush()
print(f"Producer stopped cleanly at {msg_count} messages.")


Starting producer at ~10000 msgs/sec...


In [ ]:
stop_producer = True